In [42]:
import google.auth
from google.cloud import bigquery

## Get the project_id
CREDENTIALS, PROJECT_ID = google.auth.default()
print(f"Detected Project ID: {PROJECT_ID}")

## Set various names
DATASET_ID = "bootcamp_challenge1"
RAW_TABLE = "fraud_raw_data"
MODIFIED_TABLE = "fraud_training_data"

GCS_URI = "gs://labs.roitraining.com/data-to-ai-workshop/fraud_data_raw.csv"

Detected Project ID: qwiklabs-gcp-00-c2e92c8fc9eb


In [43]:
## Connect BigQuery Client and create dataset

client = bigquery.Client(project=PROJECT_ID)

dataset_ref = bigquery.Dataset(f"{PROJECT_ID}.{DATASET_ID}")
dataset_ref.location = "US"

try:
    client.create_dataset(dataset_ref)
    print("Dataset created.")
except Exception:
    print("Dataset already exists.")

## Ingeset data
table_id = f"{PROJECT_ID}.{DATASET_ID}.{RAW_TABLE}"

job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True,
)

load_job = client.load_table_from_uri(
    GCS_URI,
    table_id,
    job_config=job_config,
)

load_job.result()
print("Raw data loaded into BigQuery.")

Dataset already exists.
Raw data loaded into BigQuery.


In [44]:
## Load data from BigQuery

import pandas as pd

query = f"""
SELECT *
FROM `{PROJECT_ID}.{DATASET_ID}.{RAW_TABLE}`
"""

df = client.query(query).to_dataframe()
df.head()

,Applicant_ID,Age,Employment_Status,Income,Number_of_Dependents,Amount_Requested,Previous_Assistance_Received,Previous_Assistance_Date,Supporting_Doc_Verified,Application_Frequency_Last_Year,IP_Address,Device_Type,Application_Date,Fraudulent
0,217,65,Unemployed,28984,4,5872,False,NaT,False,1,156.133.45.45,Mobile,2024-08-18,0
1,226,54,Self-Employed,0,1,6631,False,NaT,False,1,245.13.80.245,Tablet,2024-05-11,0
2,240,26,Self-Employed,64477,5,8612,False,NaT,True,1,213.103.170.95,Mobile,2024-08-14,0
3,252,28,Unemployed,28576,4,2951,False,NaT,True,1,234.179.149.207,Desktop,2024-06-12,0
4,266,43,Employed,44930,5,2324,False,NaT,False,1,66.109.96.227,Mobile,2024-08-16,0


In [45]:
## 3.1 One-hot encode the Employment_Status and Device_Type fields

df = pd.get_dummies(
    df,
    columns=["Employment_Status", "Device_Type"],
    prefix=["Employment_Status", "Device_Type"]
)
df.head()

,Applicant_ID,Age,Income,Number_of_Dependents,Amount_Requested,Previous_Assistance_Received,Previous_Assistance_Date,Supporting_Doc_Verified,Application_Frequency_Last_Year,IP_Address,Application_Date,Fraudulent,Employment_Status_Employed,Employment_Status_Self-Employed,Employment_Status_Unemployed,Device_Type_Desktop,Device_Type_Mobile,Device_Type_Tablet
0,217,65,28984,4,5872,False,NaT,False,1,156.133.45.45,2024-08-18,0,False,False,True,False,True,False
1,226,54,0,1,6631,False,NaT,False,1,245.13.80.245,2024-05-11,0,False,True,False,False,False,True
2,240,26,64477,5,8612,False,NaT,True,1,213.103.170.95,2024-08-14,0,False,True,False,False,True,False
3,252,28,28576,4,2951,False,NaT,True,1,234.179.149.207,2024-06-12,0,False,False,True,True,False,False
4,266,43,44930,5,2324,False,NaT,False,1,66.109.96.227,2024-08-16,0,True,False,False,False,True,False


In [46]:
## 3.2 Break the age field into bins, and one-hot encode the bins

age_bins_cutoff = [18, 25, 35, 45, 55, 120]
age_labels = ["18_24", "25_34", "35_44", "45_54", "55_plus"]

df["Age_Bins"] = pd.cut(df["Age"], bins=age_bins_cutoff, labels=age_labels, right=False)

df = pd.get_dummies(df, columns=["Age_Bins"], prefix="Age")
df.head()

,Applicant_ID,Age,Income,Number_of_Dependents,Amount_Requested,Previous_Assistance_Received,Previous_Assistance_Date,Supporting_Doc_Verified,Application_Frequency_Last_Year,IP_Address,...,Employment_Status_Self-Employed,Employment_Status_Unemployed,Device_Type_Desktop,Device_Type_Mobile,Device_Type_Tablet,Age_18_24,Age_25_34,Age_35_44,Age_45_54,Age_55_plus
0,217,65,28984,4,5872,False,NaT,False,1,156.133.45.45,...,False,True,False,True,False,False,False,False,False,True
1,226,54,0,1,6631,False,NaT,False,1,245.13.80.245,...,True,False,False,False,True,False,False,False,True,False
2,240,26,64477,5,8612,False,NaT,True,1,213.103.170.95,...,True,False,False,True,False,False,True,False,False,False
3,252,28,28576,4,2951,False,NaT,True,1,234.179.149.207,...,False,True,True,False,False,False,True,False,False,False
4,266,43,44930,5,2324,False,NaT,False,1,66.109.96.227,...,False,False,False,True,False,False,False,True,False,False


In [47]:
## 3.3 Create the new field  Income-to-Amount-Requested

df["Income_to_Amount_Requested"] = (
    df["Income"] / df["Amount_Requested"]
)

df["Income_to_Amount_Requested"] = df["Income_to_Amount_Requested"].replace([float("inf")], 0)

df.head()

,Applicant_ID,Age,Income,Number_of_Dependents,Amount_Requested,Previous_Assistance_Received,Previous_Assistance_Date,Supporting_Doc_Verified,Application_Frequency_Last_Year,IP_Address,...,Employment_Status_Unemployed,Device_Type_Desktop,Device_Type_Mobile,Device_Type_Tablet,Age_18_24,Age_25_34,Age_35_44,Age_45_54,Age_55_plus,Income_to_Amount_Requested
0,217,65,28984,4,5872,False,NaT,False,1,156.133.45.45,...,True,False,True,False,False,False,False,False,True,4.935967
1,226,54,0,1,6631,False,NaT,False,1,245.13.80.245,...,False,False,False,True,False,False,False,True,False,0.0
2,240,26,64477,5,8612,False,NaT,True,1,213.103.170.95,...,False,False,True,False,False,True,False,False,False,7.486879
3,252,28,28576,4,2951,False,NaT,True,1,234.179.149.207,...,True,True,False,False,False,True,False,False,False,9.683497
4,266,43,44930,5,2324,False,NaT,False,1,66.109.96.227,...,False,False,True,False,False,False,True,False,False,19.333046


In [48]:
## 3.4 Create a calculated field called Time_Since_Previous_Assistance

df["Application_Date"] = pd.to_datetime(df["Application_Date"])
df["Previous_Assistance_Date"] = pd.to_datetime(df["Previous_Assistance_Date"])

df["Time_Since_Previous_Assistance"] = (
    df["Application_Date"] - df["Previous_Assistance_Date"]
).dt.days

df.head()

,Applicant_ID,Age,Income,Number_of_Dependents,Amount_Requested,Previous_Assistance_Received,Previous_Assistance_Date,Supporting_Doc_Verified,Application_Frequency_Last_Year,IP_Address,...,Device_Type_Desktop,Device_Type_Mobile,Device_Type_Tablet,Age_18_24,Age_25_34,Age_35_44,Age_45_54,Age_55_plus,Income_to_Amount_Requested,Time_Since_Previous_Assistance
0,217,65,28984,4,5872,False,NaT,False,1,156.133.45.45,...,False,True,False,False,False,False,False,True,4.935967,NaN
1,226,54,0,1,6631,False,NaT,False,1,245.13.80.245,...,False,False,True,False,False,False,True,False,0.0,NaN
2,240,26,64477,5,8612,False,NaT,True,1,213.103.170.95,...,False,True,False,False,True,False,False,False,7.486879,NaN
3,252,28,28576,4,2951,False,NaT,True,1,234.179.149.207,...,True,False,False,False,True,False,False,False,9.683497,NaN
4,266,43,44930,5,2324,False,NaT,False,1,66.109.96.227,...,False,True,False,False,False,True,False,False,19.333046,NaN


In [49]:
## 3.5 Change True/False fields to 0s and 1s

bool_columns = df.select_dtypes(include="bool").columns
df[bool_columns] = df[bool_columns].astype(int)

df.head()

,Applicant_ID,Age,Income,Number_of_Dependents,Amount_Requested,Previous_Assistance_Received,Previous_Assistance_Date,Supporting_Doc_Verified,Application_Frequency_Last_Year,IP_Address,...,Device_Type_Desktop,Device_Type_Mobile,Device_Type_Tablet,Age_18_24,Age_25_34,Age_35_44,Age_45_54,Age_55_plus,Income_to_Amount_Requested,Time_Since_Previous_Assistance
0,217,65,28984,4,5872,0,NaT,0,1,156.133.45.45,...,0,1,0,0,0,0,0,1,4.935967,NaN
1,226,54,0,1,6631,0,NaT,0,1,245.13.80.245,...,0,0,1,0,0,0,1,0,0.0,NaN
2,240,26,64477,5,8612,0,NaT,1,1,213.103.170.95,...,0,1,0,0,1,0,0,0,7.486879,NaN
3,252,28,28576,4,2951,0,NaT,1,1,234.179.149.207,...,1,0,0,0,1,0,0,0,9.683497,NaN
4,266,43,44930,5,2324,0,NaT,0,1,66.109.96.227,...,0,1,0,0,0,1,0,0,19.333046,NaN


In [50]:
## Save data
table_id = f"{PROJECT_ID}.{DATASET_ID}.{MODIFIED_TABLE}"

job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE"
)

job = client.load_table_from_dataframe(
    df,
    table_id,
    job_config=job_config
)

job.result()

LoadJob<project=qwiklabs-gcp-00-c2e92c8fc9eb, location=US, id=3a7230fc-3a96-47bb-8ece-335f21b92e68>